In [318]:
import numpy as np
import bokeh
from tqdm import tqdm
import numba
colors = ['#ffffb2','#fed976','#feb24c','#fd8d3c','#f03b20','#bd0026']
# 20 viridis colors = ['#440154',"#481567","#482677","#453781","#404788","#39568c","#33638d","#2d708e","#287d8e","#238a8d","#1f968b","#20a387","#29af7f","#3cbb75","#55c667","#73d055","#95d840","#b8de29","#dce319","#fde725"]
bokeh.io.output_notebook()

Loading BokehJS ...

# Modeling 1D Oxygen Diffusion with the Forward Euler method

Diffusion of any molecule $\theta$ through a symmetrical system can be modeled in 1D using Fick's law:

\begin{align}
\frac{d\theta}{dt} = D \frac{d^2 \theta}{dx^2}
\end{align}

where $D$ is the diffusion constant. This same equation can be represented as a difference equation:

\begin{align}
\frac{\theta(x,t+\Delta t) - \theta(x,t)}{\Delta t} = D \frac{\theta(x+\Delta x, t) + \theta(x-\Delta x, t) - 2\theta(x,t)}{\Delta x^2}
\end{align}

From any arbitrary initial and boundary conditions, we can march forward in time by iteratively calculating the rearranged equation:

\begin{align}
\theta(x,t+\Delta t) = \theta(x,t) + D \Delta t \frac{\theta(x+\Delta x, t) + \theta(x-\Delta x, t) - 2\theta(x,t)}{\Delta x^2}
\end{align}

This method is called the Forward Euler method.

From Suzy and Griffin: "A major point to be wary of is the instability of this method. The error in this scales with the square of our step size. We must choose a sufficiently small step in time such that at most only one computable event must occur. For example, if we are integrating exponential growth of bacterial cells, we don't want to take time steps larger than a cell division! This requirement is known as the Courant-Friedrichs-Lewy condition (the CFL) and is important for many different time-marching computer simulations."

For our simulation, we will model oxygen's diffusion through an initially anoxic agar block that is open to ambient air on one side, and closed on the other.

### Parameters

In [332]:
# Set simulation parameters
length = 1000 # µm
dx = 1.0 # µm
duration = 10 # sec
dt = 0.0001 # sec

# Diffusion coefficient for oxygen at 25˚C in a biofilm is 2.6 * 10 ** -5 cm^2/sec (Beuling EE, Van den Heuvel JC, Ottengraf SP. Determination of biofilm diffusion coefficients using micro-electrodes. Prog Biotechnol. 1996;11:31–38.)
# multiply cm^2 by 10^8 to get µm^2
D_oxy = 2.6 * 10 ** 3 # µm^2/sec
k = D_oxy / dx # µm/sec, for CFL calculation

# Set the initial oxygen value 
c_i = 200 # µM

# Initialize data output array
n_gridspaces = int(length / dx)
n_timesteps = int(duration / dt)
c = np.zeros([n_gridspaces, n_timesteps])

# The initial oxygen level will be localized at the left boundary
c[0, 0] = c_i

# CFL check
CFL = k * dt / dx
print("CFL = "+str(CFL))
if not CFL < 0.5:
    print("Warning: Parameter values are nearing violation of the Courant–Friedrichs–Lewy condition. k * dt / dx must be less than 0.5 or instabilities in the simulation will likely occur.")

CFL = 0.26


In [333]:
if not CFL <= 1.0:
    raise RuntimeError("Warning: Parameter values violate the Courant–Friedrichs–Lewy condition. k * dt / dx must be less than or equal to 1.0 or instabilities in the simulation will occur.")

# timestepping
for t in tqdm(range(1, n_timesteps)):
    
    for x in range(1, n_gridspaces-1):
        
        # update non-boundaries
        c[x,t] = c[x,t-1] + (D_oxy*dt/dx**2) * (c[x-1,t-1] + c[x+1,t-1] - 2*c[x,t-1])
        
    # update left boundary
    c[0,t] = c_i
                
    # update right boundary
    c[-1, t] = c[-1,t-1] + (D_oxy*dt/dx**2) * (c[-2,t-1] - c[-1,t-1])
    
output = c

100%|██████████| 99999/99999 [06:32<00:00, 254.81it/s]


In [334]:
# plot the results
p = bokeh.plotting.figure(
    height=350,
    width=550,
    x_axis_label='x (µm)',
    y_axis_label='O2 (µM)',
    background_fill_color='black'
)

x = np.arange(n_gridspaces)*dx
n_timeslices = int(n_timesteps / (len(colors)-1))

for step, color in zip(range(0, n_timesteps, n_timeslices), colors):
    p.line(
        x=x,
        y=output[:,step],
        line_join='bevel',
        line_width=4,
        color=color,
        legend_label=str(round(step*dt, 2))+' sec',
    )

# plot the final timepoint along with the others captured in the for loop
p.line(
        x=x,
        y=output[:,-1],
        line_join='bevel',
        line_width=4,
        color=colors[-1],
        legend_label=str(duration)+' sec',
    )
                       
bokeh.io.show(p)